## Database Creation

In [1]:
# Import sqlite3 module into
# this program as sq
import sqlite3 as sq
 
# Import pandas module into
# this program as pd
import pandas as pd
   
# Create a connection object,
# Make a new db if not exist already 
# and connect it, if exist then connect.
connection = sq.connect('ClinicalData.db')
 
# Create a cursor object
curs = connection.cursor()

In [3]:
# Run create table sql query
curs.execute('''CREATE TABLE PatientInfo(desysortkey, genderid, Diabetes, COPD, CKD, PVD, CAD, HyperTension, DiseaseType, Age, race, Treated, HemoglobinA1C, RandomGlucose, FastingGlucose, BMI, SystolicBP, DiastolicBP, HDLC, LDLC, TotalCholesterol, Triglycerides, COPDAssesstmetTest, nMRC, FEV1FVC_Ratio, FEV1, serumcreatinine, gfr, bloodureanitrogen, Albumintocreatinine_ratio)''')


In [4]:
# Load CSV data into Pandas DataFrame
student = pd.read_csv('Data.csv')
 
# Write the data to a sqlite db table
student.to_sql('PatientInfo', connection, if_exists='replace', index=False)
   
# Run select sql query
curs.execute('select * from PatientInfo')
 
# Fetch all records
# as list of tuples
#records = curs.fetchall()
 
# Display result 
#for row in records:
    # show row
    #print(row)
     
# Close connection to SQLite database
#connection.close()

## Implementation Of Web App

In [8]:
from flask import *
import pandas as pd
import json
from scipy.stats import mode
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib
matplotlib.use('Agg')
from io import BytesIO
import base64
import pickle
import numpy as np





%matplotlib inline

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def login():
    error = None
    if request.method == 'POST':
        if request.form['username'] != 'admin' or request.form['password'] != 'admin@123':
            error = 'Invalid Credentials. Please try again.'
        else:
            return redirect(url_for('patientassesment'))
    return render_template('login.html', error=error)

def get_db_connection():
    connection = sq.connect('ClinicalData.db')
 
    # Create a cursor object
    curs = connection.cursor()
    return connection

@app.route('/patientassesment')

def patientassesment():
    return render_template('patientassessment.html')

@app.route('/patientnewrecored')
def patientnewrecored():
    return render_template('patientnewrecored.html')

@app.route('/newpatient',methods=['GET', 'POST'])
def newpatient():
    return render_template('Newpatient.html')
 
    
def diapredict(val,dic):
    model = pickle.load(open('diabetes.pkl','rb'))
    values = np.asarray(val)
    diamodel=model.predict(values.reshape(1, -1))[0]
    if diamodel == 2 :
        msg1="No Risk"
    elif diamodel == 1 :
        msg1="Early stage"
    else:
        msg1="Critical"
    return msg1
    
def copdpredict(val,dic):
    model = pickle.load(open('copd.pkl','rb'))
    values = np.asarray(val)
    copdmodel=model.predict(values.reshape(1, -1))[0]
    if copdmodel == 2 :
        msg="No Risk"
    elif copdmodel == 1 :
        msg="Early stage"
    else:
        msg="Critical"
    return msg
    
def ckdpredict(val,dic):
    model = pickle.load(open('ckd.pkl','rb'))
    values = np.asarray(val)
    ckdmodel=model.predict(values.reshape(1, -1))[0]
    if ckdmodel == 2 :
        msg3="No Risk"
    elif ckdmodel == 1 :
        msg3="Early stage"
    else:
        msg3="Critical"
    return msg3


def cadpredict(val,dic):
    model = pickle.load(open('cad.pkl','rb'))
    values = np.asarray(val)
    cadmodel=model.predict(values.reshape(1, -1))[0]
    if cadmodel == 2 :
        msg4="No Risk"
    elif cadmodel == 1 :
        msg4="Early stage"
    else:
        msg4="Critical"
    return msg4 
    
def hyperpredict(val,dic):
    model = pickle.load(open('hypertension.pkl','rb'))
    values = np.asarray(val)
    hypermodel=model.predict(values.reshape(1, -1))[0]
    if hypermodel == 2 :
        msg2="No Risk"
    elif hypermodel == 1 :
        msg2="Early stage"
    else:
        msg2="Critical"
    return msg2 
    
    
    
@app.route("/predict", methods = ['POST', 'GET'])
def predictPage():
        if request.method == 'POST':
            to_predict_dict = request.form.to_dict()

            to_predict_list = list(map(float, list(to_predict_dict.values())))
                
            diapred = diapredict(to_predict_list,to_predict_dict)
            copdpred=copdpredict(to_predict_list,to_predict_dict)
            ckdpred=ckdpredict(to_predict_list,to_predict_dict)
            cadpred=cadpredict(to_predict_list,to_predict_dict)
            hyperpred=hyperpredict(to_predict_list,to_predict_dict)
            return render_template('predict.html', diapred=diapred,copdpred=copdpred,ckdpred=ckdpred,cadpred=cadpred,hyperpred=hyperpred)    
                            


@app.route('/usersearch',methods = ['POST','GET'])  
def usersearch():
    if request.method == 'POST':
        userid=request.form["patientid"]
        return redirect(url_for('GetPatientDetails',userid=userid))

    
@app.route('/fetchPatientParams/<int:userid>',methods = ['GET'])
def fetchPatientParams(userid):
    if request.method == 'GET':
        connection=get_db_connection()
        df = pd.read_sql_query("SELECT * from PatientInfo", connection)
        #patdetails=pd.DataFrame(df.loc[df["desy_sort_key"]==userid] [['desy_sort_key','gender_id','Age']]).to_dict('records')
        #userdetails=json.dumps(patdetails)
        details=pd.DataFrame(df.loc[df["desy_sort_key"]==userid] )
        details=details.drop(columns=['desy_sort_key','Unnamed: 0'],axis=1).to_dict('records')
        
        return details
    
    
@app.route('/GetPatientDetails/<int:userid>',methods = ['POST','GET'])
def GetPatientDetails(userid):
    if request.method == 'GET':
        connection=get_db_connection()
        df = pd.read_sql_query("SELECT * from PatientInfo", connection)
        patdetails=pd.DataFrame(df.loc[df["desy_sort_key"]==userid] [['desy_sort_key','gender_id','Age']]).to_dict('records')
        #patdetails=pd.DataFrame(data.loc[data["desy_sort_key"]==userid] [['desy_sort_key','gender_id','Age']]).to_dict('records')
        userdetails=json.dumps(patdetails)
        details=pd.DataFrame(df.loc[df["desy_sort_key"]==userid])
        #details=pd.DataFrame(data.loc[data["desy_sort_key"]==userid] )
        details=details.drop(columns=['desy_sort_key','gender_id','DiseaseType','race','Treated','Unnamed: 0'],axis=1).to_dict('records')
        patientcomponent=list(details[0].values())
        patientcomponents=json.dumps(patientcomponent)
        return redirect(url_for('DisplayPatientDetails',userdetails=userdetails,patientcomponents=patientcomponents))
  

@app.route('/DisplayPatientDetails/<userdetails>/<patientcomponents>',methods = ['POST','GET'])
def DisplayPatientDetails(userdetails,patientcomponents):
    if request.method == 'GET':
        usd=json.loads(userdetails)
        patcomponents=json.loads(patientcomponents)
        model = pickle.load(open('copd.pkl','rb'))
        values = np.asarray(patcomponents)
        copdmodel=model.predict(values.reshape(1, -1))[0]
        if copdmodel == 2 :
            msg="No Risk"
        elif copdmodel == 1 :
            msg="Early stage"
        else:
            msg="Critical"
        model = pickle.load(open('diabetes.pkl','rb'))
        values = np.asarray(patcomponents)
        diamodel=model.predict(values.reshape(1, -1))[0]
        if diamodel == 2 :
            msg1="No Risk"
        elif diamodel == 1 :
            msg1="Early stage"
        else:
            msg1="Critical"
        model = pickle.load(open('hypertension.pkl','rb'))
        values = np.asarray(patcomponents)
        hypermodel=model.predict(values.reshape(1, -1))[0]
        if hypermodel == 2 :
            msg2="No Risk"
        elif hypermodel == 1 :
            msg2="Early stage"
        else:
            msg2="Critical"
        model = pickle.load(open('ckd.pkl','rb'))
        values = np.asarray(patcomponents)
        ckdmodel=model.predict(values.reshape(1, -1))[0]
        if ckdmodel == 2 :
            msg3="No Risk"
        elif ckdmodel == 1 :
            msg3="Early stage"
        else:
            msg3="Critical"
        model = pickle.load(open('cad.pkl','rb'))
        values = np.asarray(patcomponents)
        cadmodel=model.predict(values.reshape(1, -1))[0]
        if cadmodel == 2 :
            msg4="No Risk"
        elif cadmodel == 1 :
            msg4="Early stage"
        else:
            msg4="Critical"
    return render_template("PatientDetails.html",usd=usd,msg=msg,msg1=msg1,msg2=msg2,msg3=msg3,msg4=msg4)



if __name__ =='__main__':
    app.run() 

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/css/bootstrap.min.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/css/font-awesome.min.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/css/style2.css HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/js/popper.min.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/js/jquery-3.2.1.min.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/js/bootstrap.min.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/js/app.js HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:44] "GET /static/img/logo-dark.png HTTP/1.1" 304 -
127.0.0.1 - - [14/Apr/2023 13:07:50] "POST / HTTP/1.1" 302 -
127.0.0.1 - - [14/Apr/2023 13:07:50] "GET /patientassesment HTTP/1.1" 200 -
127.0.0.1 - - [14/Apr/2023 13:07:50] "GET /static/css/bootstrap.min.